<a href="https://colab.research.google.com/github/hossamEldeinMohmed/Data-VisliaZation-Project/blob/main/Practical_PySpark_DF_SQL_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Labs 1 and 2 PySpark:**

In these labs we will be using the "[[NeurIPS 2020] Data Science for COVID-19 (DS4C)](https://www.kaggle.com/datasets/kimjihoo/coronavirusdataset?select=PatientInfo.csv)" dataset, retrieved from [Kaggle](https://www.kaggle.com/) on 1/6/2022, for educational non commercial purpose, License
[CC BY-NC-SA 4.0
](https://creativecommons.org/licenses/by-nc-sa/4.0/)


The csv file that we will be using in this lab is **PatientInfo**.

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 45.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=b178c473ec663239d7ff50c88876436a9b1e3ec5570b1a1590d582afe6bf12fd
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


## PatientInfo.csv

**patient_id**
the ID of the patient

**sex**
the sex of the patient

**age**
the age of the patient

**country**
the country of the patient

**province**
the province of the patient

**city**
the city of the patient

**infection_case**
the case of infection

**infected_by**
the ID of who infected the patient


**contact_number**
the number of contacts with people

**symptom_onset_date**
the date of symptom onset

**confirmed_date**
the date of being confirmed

**released_date**
the date of being released

**deceased_date**
the date of being deceased

**state**
isolated / released / deceased

### Import the pyspark and check it's version

In [ ]:
#Import the pyspark and check it's version
import pyspark
print("PySpark version:", pyspark.__version__)


PySpark version: 3.3.0


### Import and create SparkSession

In [ ]:
#Create a SparkSession
spark = pyspark.sql.SparkSession.builder.getOrCreate()


### Load the PatientInfo.csv file and show the first 5 rows

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
# Load the PatientInfo.csv file and show the first 5 rows
patient_info = spark.read.option("header", "true").csv(
    "/content/PatientInfo.csv", inferSchema=True)
patient_info.show(5)


+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|

### Display the schema of the dataset

In [ ]:
# Display the schema of the dataset
patient_info.printSchema()


root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: timestamp (nullable = true)
 |-- released_date: timestamp (nullable = true)
 |-- deceased_date: timestamp (nullable = true)
 |-- state: string (nullable = true)



### Display the statistical summary

In [ ]:
# Display the statistical summary
patient_info.describe().show()


+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------+
|summary|          patient_id|   sex| age|   country|province|          city|      infection_case|         infected_by|      contact_number|symptom_onset_date|   state|
+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------+
|  count|                5165|  4043|3785|      5165|    5165|          5071|                4246|                1346|                 791|               690|    5165|
|   mean|2.8636345618679576E9|  null|null|      null|    null|          null|                null|2.2845944015643125E9|1.6772572523506988E7|              null|    null|
| stddev| 2.074210725277473E9|  null|null|      null|    null|          null|                null|1.5265072953383324E9| 3.093097580985502E8|              n

### Using the state column.
### How many people survived (released), and how many didn't survive (isolated/deceased)?

In [ ]:
# Using the state column.
# How many people survived (released), and how many didn't survive (isolated/deceased)?
patient_info.groupBy("State").count().show()



+--------+-----+
|   State|count|
+--------+-----+
|isolated| 2158|
|released| 2929|
|deceased|   78|
+--------+-----+



### Display the number of null values in each column

In [ ]:
from string import *
from pyspark.sql.functions import *


In [ ]:
# Display the number of null values in each column
from pyspark.sql.functions import col, isnan, when, count
from pyspark.sql.functions import *
patient_info.select([count(when(col(c).isNull(),c)).alias(c+" Nulls") for c in patient_info.columns]).show()


+----------------+---------+---------+-------------+--------------+----------+--------------------+-----------------+--------------------+------------------------+--------------------+-------------------+-------------------+-----------+
|patient_id Nulls|sex Nulls|age Nulls|country Nulls|province Nulls|city Nulls|infection_case Nulls|infected_by Nulls|contact_number Nulls|symptom_onset_date Nulls|confirmed_date Nulls|released_date Nulls|deceased_date Nulls|state Nulls|
+----------------+---------+---------+-------------+--------------+----------+--------------------+-----------------+--------------------+------------------------+--------------------+-------------------+-------------------+-----------+
|               0|     1122|     1380|            0|             0|        94|                 919|             3819|                4374|                    4475|                   3|               3578|               5099|          0|
+----------------+---------+---------+-------------+

## Data preprocessing

### Fill the nulls in the deceased_date with the released_date. 
- You can use <b>coalesce</b> function

In [ ]:
#import coalesce 
from pyspark.sql.functions import coalesce


In [ ]:
# Fill the nulls in the deceased_date with the released_date.
#You can use coalesce function
patient_info = patient_info.withColumn("deceased_date", coalesce(
    patient_info.deceased_date, patient_info.released_date))
patient_info.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|      deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|2020-02-05 00:00:00|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|2020-03-02 00:00:00|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-

### Add a column named no_days which is difference between the deceased_date and the confirmed_date then show the top 5 rows. Print the schema.
- <b> Hint: You need to typecast these columns as date first <b>

In [ ]:
# Add a column named no_days which is difference between the deceased_date and the confirmed_date then show the top 5 rows. Print the schema.
# Hint: You need to typecast these columns as date first 
patient_info = patient_info.withColumn("no_days", (patient_info.deceased_date - patient_info.confirmed_date).cast("int"))
patient_info.show(5)
patient_info.printSchema()

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|      deceased_date|   state|no_days|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|2020-02-05 00:00:00|released|1123200|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|2020-03-02 00:00:00|released|2764800|
|1000000003|  m

In [ ]:
#import sparck function
from pyspark.sql.functions import *

# Add a is_male column if male then it should yield true, else then False

In [ ]:
# Add a is_male column if male then it should yield true, else then False
patient_info = patient_info.withColumn('is_male',when((patient_info["sex"] == "male"), True).otherwise(False))

### Add a is_dead column if patient state is not released then it should yield true, else then False

- Use <b>UDF</b> to perform this task. 
- However, UDF is not recommended there is no built in function can do the required operation.
- UDF is slower than built in functions.

In [ ]:
# Add a is_dead column if patient state is not released then it should yield true, else then False

#Use UDF to perform this task. 
#However, UDF is not recommended there is no built in function can do the required operation.
# UDF is slower than built in functions.
# You can use the following code to create a UDF
# You can use the following code to create 
from pyspark.sql.functions import udf
is_dead_udf = udf(lambda x: True if x == "isolated" or x == "deceased" else False)
patient_info = patient_info.withColumn("is_dead", is_dead_udf(patient_info.state))
patient_info.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+-------+-------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|      deceased_date|   state|no_days|is_male|is_dead|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+-------+-------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|2020-02-05 00:00:00|released|1123200|   true|  false|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-0

### Change the ages to bins from 10s, 0s, 10s, 20s,.etc to 0,10, 20

In [ ]:
# Change the ages to bins from 10s, 0s, 10s, 20s,.etc to 0,10, 20
patient_info = patient_info.withColumn("age", regexp_replace("age", "s", ''))
patient_info.show(2)

+----------+----+---+-------+--------+-----------+---------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+-------+-------+
|patient_id| sex|age|country|province|       city| infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|      deceased_date|   state|no_days|is_male|is_dead|
+----------+----+---+-------+--------+-----------+---------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+-------+-------+
|1000000001|male| 50|  Korea|   Seoul| Gangseo-gu|overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|2020-02-05 00:00:00|released|1123200|   true|  false|
|1000000002|male| 30|  Korea|   Seoul|Jungnang-gu|overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|2020-03-02 00:00:00|rele

### Change age, and no_days  to be typecasted as Doubleis_male

In [ ]:
# Change age, and no_days  to be typecasted as Double
patient_info = patient_info.withColumn("age", patient_info.age.cast("double"))
patient_info = patient_info.withColumn("no_days", patient_info.no_days.cast("double"))


In [ ]:
patient_info.show(5)

+----------+------+----+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+---------+-------+-------+
|patient_id|   sex| age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|      deceased_date|   state|  no_days|is_male|is_dead|
+----------+------+----+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+---------+-------+-------+
|1000000001|  male|50.0|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|2020-02-05 00:00:00|released|1123200.0|   true|  false|
|1000000002|  male|30.0|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00

### Drop the columns
["patient_id","sex","infected_by","contact_number","released_date","state",
"symptom_onset_date","confirmed_date","deceased_date","country","no_days",
"city","infection_case"]

In [ ]:
# Drop the columns
#["patient_id","sex","infected_by","contact_number","released_date","state","symptom_onset_date","confirmed_date","deceased_date","country","no_days","city","infection_case"]

patient_info=patient_info.drop("patient_id","sex","infected_by","contact_number","released_date","state",
"symptom_onset_date","confirmed_date","deceased_date","country","no_days",
"city","infection_case") 

### Recount the number of nulls now

In [ ]:
# count the number of nulls 
patient_info.select([count(when(col(c).isNull(),c)).alias(c+" Nulls") for c in patient_info.columns]).show()

+---------+--------------+-------------+-------------+
|age Nulls|province Nulls|is_male Nulls|is_dead Nulls|
+---------+--------------+-------------+-------------+
|     1380|             0|            0|            0|
+---------+--------------+-------------+-------------+



In [ ]:
patient_info.show()

+----+--------+-------+-------+
| age|province|is_male|is_dead|
+----+--------+-------+-------+
|50.0|   Seoul|   true|  false|
|30.0|   Seoul|   true|  false|
|50.0|   Seoul|   true|  false|
|20.0|   Seoul|   true|  false|
|20.0|   Seoul|  false|  false|
|50.0|   Seoul|  false|  false|
|20.0|   Seoul|   true|  false|
|20.0|   Seoul|   true|  false|
|30.0|   Seoul|   true|  false|
|60.0|   Seoul|  false|  false|
|50.0|   Seoul|  false|  false|
|20.0|   Seoul|   true|  false|
|80.0|   Seoul|   true|   true|
|60.0|   Seoul|  false|  false|
|70.0|   Seoul|   true|  false|
|70.0|   Seoul|   true|  false|
|70.0|   Seoul|   true|  false|
|20.0|   Seoul|   true|  false|
|70.0|   Seoul|  false|  false|
|70.0|   Seoul|  false|  false|
+----+--------+-------+-------+
only showing top 20 rows



## Now do the same but using SQL select statement

### From the original Patient DataFrame, Create a temporary view (table).

In [ ]:
df2= spark.read.option("header", "true").csv(
    "PatientInfo.csv", inferSchema=True)

In [ ]:
# From the original Patient DataFrame, Create a temporary view (table).
df2.createOrReplaceTempView("patient_info")


### Use SELECT statement to select all columns from the dataframe and show the output.

In [ ]:
# Use SELECT statement to select all columns from the dataframe and show the
spark.sql("SELECT * FROM patient_info ").show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 20020

### *Using SQL commands*, limit the output to only 5 rows 

In [ ]:
# *Using SQL commands*, limit the output to only 5 rows 
spark.sql("SELECT * FROM patient_info LIMIT 5")

DataFrame[patient_id: bigint, sex: string, age: string, country: string, province: string, city: string, infection_case: string, infected_by: string, contact_number: string, symptom_onset_date: string, confirmed_date: timestamp, released_date: timestamp, deceased_date: timestamp, state: string]

### Select the count of males and females in the dataset

In [ ]:
# Select the count of males and females in the patient_info
spark.sql("SELECT state, COUNT(patient_id)\
            from patient_info\
                group by state").show()

+--------+-----------------+
|   state|count(patient_id)|
+--------+-----------------+
|isolated|             2158|
|released|             2929|
|deceased|               78|
+--------+-----------------+



### How many people did survive, and how many didn't?

In [ ]:
# How many people did survive, and how many didn't?
spark.sql("SELECT  COUNT(patient_id)\
             from patient_info\
                 group by sex").show()


+-----------------+
|count(patient_id)|
+-----------------+
|             1122|
|             2218|
|             1825|
+-----------------+



### Now, let's perform some preprocessing using SQL:
1. Convert *age* column to double after removing the 's' at the end -- *hint: check SUBSTRING method*
2. Select only the following columns: `['sex', 'age', 'province', 'state']`
3. Store the result of the query in a new dataframe

## Machine Learning 
### Create a pipeline model to predict is_dead and evaluate the performance.
- Use <b>StringIndexer</b> to transform <b>string</b> data type to indices.
- Use <b>OneHotEncoder</b> to deal with categorical values.
- Use <b>Imputer</b> to fill missing data with mean.

In [ ]:
import numpy as np

In [ ]:

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline



In [ ]:
patient_info.printSchema()


root
 |-- age: double (nullable = true)
 |-- province: string (nullable = true)
 |-- is_male: boolean (nullable = false)
 |-- is_dead: string (nullable = true)



In [ ]:
patient_info.withColumn('is_male', (patient_info.is_male).cast('integer')) 

DataFrame[age: double, province: string, is_male: int, is_dead: string]

In [ ]:
#Use StringIndexer to transform string data type to indices.
from pyspark.ml.feature import Imputer, StringIndexer, VectorAssembler
from pyspark.ml.feature import OneHotEncoder

index = StringIndexer(inputCols=["is_dead","province"], outputCols=["is_dead_index","province_index"],handleInvalid="skip")
ohe = OneHotEncoder(inputCols=["age","province_index"], outputCols=["age_index_","province_index_"])

rfc = RandomForestClassifier(featuresCol="features",predictionCol="is_dead")
model = Pipeline(stages=[index,ohe,rfc]).fit(patient_info)
